# Import modules

In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install re


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time
import threading
import queue
import IPython
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Navigating HTML Structures of One link

### Request and scrape the html file using beautilful soup and requests modules

In [14]:
ansem_news = 'https://coinpaper.com/5173/ansem-crypto-influence-impact-and-controversies-in-cryptocurrency' # you can follow the link if you want

# Get the contents of one dota article
response = requests.get(ansem_news)

# If scraping is allowed, parse the html document
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'lxml')
    print('Scraped')
else:
    raise Exception('Unable to scrape. Status code: {}'.format(response.status_code))
    # status code 403 means forbidden—you don't have permission to access this resource

Scraped


### Scraping DOTA 2 News Data

#### Parse the whole html file

In [15]:
html_content = soup
print(html_content)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_16,w_16/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/><link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_32,w_32/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/><link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_96,w_96/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/><link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_180,w_180/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/><title>Ansem Crypto: Influence, Impact, and Controversies in Cryptocurrency</title><meta content="index,follow,max-image-preview:large" name="robots"/><meta content="index,follow,max-image-preview:large" name="googlebot"/><meta content="Discover Ansem Crypto, examining his purpose, contribution to the cryptocurrency

In [16]:
# Print the entire HTML content to see if the page was loaded correctly
print(soup.prettify())


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_16,w_16/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/>
  <link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_32,w_32/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/>
  <link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_96,w_96/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/>
  <link href="https://res.coinpaper.com/dds1hga1g/image/upload/c_fill,h_180,w_180/favicon_605e82e261_4d8e57334e.png" rel="icon" type="image/png"/>
  <title>
   Ansem Crypto: Influence, Impact, and Controversies in Cryptocurrency
  </title>
  <meta content="index,follow,max-image-preview:large" name="robots"/>
  <meta content="index,follow,max-image-preview:large" name="googlebot"/>
  <meta content="Discover Ansem Crypto, examining his purp

#### Parse the news title using 'find'

In [17]:
# Extract all the text from the page
all_text = soup.get_text(separator='\n', strip=True)

# Print the extracted text
print(all_text)


Ansem Crypto: Influence, Impact, and Controversies in Cryptocurrency
News
All News
Business
Technology
DeFi
NFT
Bitcoin
Ethereum
Layer 2: Optimism
Altcoins
Legal
Metaverse
Crypto
Blockchain
Shiba Inu
Stories
All Stories
Markets
People
Crypto
Startups
Altcoins
Business
Technology
Explained
Legal
NFT
Trading
Bitcoin
Misc
All Misc
Reviews
Events
Jobs
Crypto
Learn
All Learn
Basics
How to...
Explained
Trading
Security
Altcoins
Blockchain
Reviews
People
Ethereum
Crypto
Collections
Search
Learn
/
Media
Ansem Crypto: Influence, Impact, and Controversies in Cryptocurrency
Discover Ansem Crypto, examining his purpose, contribution to the cryptocurrency market, and controversies around him.
Riley Asak
11 min read
21 August 2024
In This Article
Early Career and Entry into Cryptocurrency
Rise to Prominence as 'The Solana Guy'
Technical Expertise in Blockchain Technology
Social Media Influence and Community Building
Strategic Partnerships and Collaborations
Investments and Venture Capital Activities

In [18]:
# Extract all the text from the page
all_text = soup.get_text(separator=' ', strip=True)

# Split the text into sentences
sentences = all_text.split('. ')

# Combine every two sentences into a single string
combined_sentences = ['. '.join(sentences[i:i+2]) for i in range(0, len(sentences), 2)]

# Create a DataFrame
df = pd.DataFrame(combined_sentences, columns=['Content'])

# Save the DataFrame to a CSV file
df.to_csv('ansem_content.csv', index=False)

# Optionally, print a message to confirm
print("DataFrame has been saved to 'ansem_content.csv'.")

DataFrame has been saved to 'ansem_content.csv'.


In [19]:
df

,Content
0,"Ansem Crypto: Influence, Impact, and Controver..."
1,Riley Asak 11 min read 21 August 2024 In This ...
2,His expertise has positioned him as a go-to vo...
3,Ansem's rise to prominence is marked by his pr...
4,His perspectives have not only garnered a sign...
...,...
68,"Ansem frequently engages with his followers, o..."
69,"Crypto journalist @InvestExpert noted, ""Ansem ..."
70,This philosophy extends to his public discussi...
71,Top read News / Crypto Bitcoin's Future in Que...


#### Parse news content in the 'div' using 'find'

In [20]:
import pandas as pd

# Read the CSV file
csv_file_path = '/content/ansem_content.csv'
df = pd.read_csv(csv_file_path)

# Drop empty rows (if any)
df.dropna(how='all', inplace=True)

# Print column names for debugging
print("Column names:")
print(df.columns)

# Convert the DataFrame to a .txt file with numbering and include header
txt_file_path = csv_file_path.replace('.csv', '.txt')

with open(txt_file_path, 'w', newline='\n') as txt_file:
    # Write the header
    txt_file.write("Index\tContent\n")
    for i, row in df.iterrows():
        content = row.get('Content', 'No Content Found')  # Ensure 'Content' column is used
        txt_file.write(f"{i + 1}\t{content}\n")

print(f"File has been converted to {txt_file_path} with row numbering and empty rows removed.")


Column names:
Index(['Content'], dtype='object')
File has been converted to /content/ansem_content.txt with row numbering and empty rows removed.


In [21]:
import pandas as pd
import re

# Define a list of specific keywords related to your topic
keywords_list = [
    'crypto', 'Solana', 'expertise', 'cryptocurrency', 'contribution',
    'decentralized', 'high risk', 'ledger', 'investing', 'WIF', 'nft', 'dao',
    'educational outreach', 'advocacy', 'solidity', 'protocol', 'leader',
    'wallet', 'cryptographic', 'solana developments', 'decentralized finance'
]

def extract_relevant_keywords(text, keywords):
    # Convert text to lowercase and split into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Find keywords present in the text
    found_keywords = [keyword for keyword in keywords if keyword in words]
    return ', '.join(found_keywords)

# Read the CSV file
csv_file_path = '/content/ansem_content.csv'
df = pd.read_csv(csv_file_path)

# Drop empty rows (if any)
df.dropna(how='all', inplace=True)

# Add a new column for relevant keywords
df['Keywords'] = df['Content'].apply(lambda x: extract_relevant_keywords(x, keywords_list))

# Save the updated DataFrame to a new CSV file
updated_csv_file_path = '/content/ansem_content_with_keywords.csv'
df.to_csv(updated_csv_file_path, index=False)

# Convert the DataFrame to a .txt file with two columns: Content and Keywords
txt_file_path = updated_csv_file_path.replace('.csv', '.txt')

with open(txt_file_path, 'w', newline='\n') as txt_file:
    # Write the header
    txt_file.write("Content\tKeywords\n")
    for i, row in df.iterrows():
        content = row.get('Content', 'No Content Found')
        keywords = row.get('Keywords', 'No Keywords Found')
        txt_file.write(f"{content}\t{keywords}\n")

print(f"File has been converted to {txt_file_path} with Content and Keywords columns.")


File has been converted to /content/ansem_content_with_keywords.txt with Content and Keywords columns.


In [22]:
df

,Content,Keywords
0,"Ansem Crypto: Influence, Impact, and Controver...","crypto, cryptocurrency, contribution, nft"
1,Riley Asak 11 min read 21 August 2024 In This ...,"crypto, expertise, cryptocurrency"
2,His expertise has positioned him as a go-to vo...,expertise
3,Ansem's rise to prominence is marked by his pr...,crypto
4,His perspectives have not only garnered a sign...,
...,...,...
68,"Ansem frequently engages with his followers, o...",
69,"Crypto journalist @InvestExpert noted, ""Ansem ...",crypto
70,This philosophy extends to his public discussi...,crypto
71,Top read News / Crypto Bitcoin's Future in Que...,crypto


# Ignore this Step

In [23]:
# import pandas as pd

# # Read the CSV file
# csv_file_path = '/content/Ansem - Controversies & News.csv'
# data = pd.read_csv(csv_file_path)

# # Drop rows where all elements are NaN or empty strings
# data.replace('', float('NaN'), inplace=True)  # Replace empty strings with NaN
# data.dropna(how='all', inplace=True)

# # Convert the DataFrame to a string and save it to a .txt file with numbering
# txt_file_path = csv_file_path.replace('.csv', '.txt')

# with open(txt_file_path, 'w', newline='\n') as txt_file:
#     for i, row in enumerate(data.iterrows(), 1):
#         # Filter out empty cells in each row
#         row_data = '\t'.join(map(str, filter(lambda x: pd.notna(x) and x != '', row[1].values)))
#         txt_file.write(f"{i}\t{row_data}\n")

# print(f"File has been converted to {txt_file_path} with row numbering and empty rows removed.")


In [24]:
# import requests
# from bs4 import BeautifulSoup
# import re

# # URL of the page to scrape
# ansem_news = 'https://hakkamukka.com/ansems-net-worth/'

# # Send a GET request to the webpage
# response = requests.get(ansem_news)

# # Check if the request was successful
# if response.status_code == 200:
#     # Parse the HTML content using BeautifulSoup
#     soup = BeautifulSoup(response.text, 'lxml')

#     # Find the content div by id
#     content_div = soup.find('div', id='bodyContent')

#     if content_div:
#         # Extract the text and clean it up
#         text = content_div.get_text(separator='\n').strip()  # Use '\n' to keep paragraphs separated
#         lines = text.split('\n')  # Split text into lines

#         # Define keywords for unwanted lines and titles
#         unwanted_keywords = ['from the kingdom hearts wiki', 'jump to navigation', 'jump to search']

#         # Filter out lines that are section titles or empty
#         filtered_lines = []
#         for line in lines:
#             # Remove unwanted line numbers and section titles
#             if not line.strip() or any(keyword in line.lower() for keyword in unwanted_keywords):
#                 continue
#             # Remove any unwanted characters or formatting
#             cleaned_line = line.strip()
#             if cleaned_line:  # Ensure line is not empty after cleaning
#                 filtered_lines.append(cleaned_line)

#         # Select lines starting from index 60 onward
#         start_index = 60
#         selected_lines = filtered_lines[start_index:]

#         # Combine selected lines into a single paragraph
#         combined_paragraph = ' '.join(selected_lines)

#         # Convert the paragraph into one sentence per line
#         sentences = re.split(r'(?<=[.!?])\s+', combined_paragraph)  # Split on sentence-ending punctuation followed by a space

#         # Save the sentences to a .txt file with one sentence per line
#         output_file = 'ansem_seeker_of_darkness.txt'
#         with open(output_file, 'w', encoding='utf-8') as file:
#             for sentence in sentences:
#                 file.write(sentence.strip() + '\n')

#         print(f'Text successfully extracted and saved to {output_file}')
#     else:
#         print('Content div not found.')
# else:
#     raise Exception(f'Unable to scrape. Status code: {response.status_code}')
